The loop traverse network shown in figure below is a simple network with initial bearing of line P–Q fixed as 79° 37ʹ 22ʺ and coordinates of fixed-point P given as E = 1000m and N = 1150m. Table below indicates network distance and interior angle measurements made with a total station instrument having HA SD of ±3ʺ and Distance SD of 2mm + 2ppm. The angular error was computed as ±4ʺ for a set of horizontal angular measurement at each station; the horizontal distance SDs are derived from distance specification with assumption that instrument and targets can be centered to an accuracy of ±0.8mm each. Adjust the measurements by transit method. Answer the following:\
(a) Adjust the measurements by least squares method to get adjusted coordinates of Q, R and S.\
(b) The standard deviation of adjusted coordinates of points Q, R and S.\
(c) The adjusted angle and distance observations.\
(d) The aposteriori variance factor.

<div align="left">

|Figure|Observations|Coordinates|
|:-:|:-:|:-:|
|<img src="images/fig1a.png" width="650">|<img src="images/fig2.png" width="300">|<img src="images/fig3.png" width="350">

</div>

In [8]:
# Import necessary libraries
from math import pi as pi, radians as rad, degrees as deg
from sympy import sqrt, atan, symbols, Matrix, diag

def degtodms(decimal):
    """Convert decimal degrees to DMS string"""
    degrees = int(decimal)
    minutes = int((decimal - degrees) * 60)
    seconds = (decimal - degrees - minutes / 60) * 3600
    return f"{degrees}° {minutes}' {seconds:.0f}\""

def traverse(txq, tyq, txr, tyr, txs, tys, max_iterations=4, max_correction=0.0010):
    # Initialize variables to store the final values
    a = None; l = None; w = None; p = None
    n = None; d = None; df = None
    
    # Given observations
    angqq = rad(119+44/60+50/3600)
    angrr = rad(45+10/3600)
    angss = rad(130+35/60+50/3600)
    angpp = rad(64+39/60+20/3600)
    dqrr = 223.600; drss = 237.184
    dspp = 145.775; dpqq = 180.278
    dirpqq = rad(79+37/60+22/3600)
    
    for iteration in range(max_iterations):
        # Parametric model equations l = f(x)
        xp, yp, xq, yq, xr, yr, xs, ys = symbols('xp, yp, xq, yq, xr, yr, xs, ys')
        angq  = atan((xr-xq)/(yr-yq)) - (atan((xp-xq)/(yp-yq)) + pi) + 2*pi
        angr = (atan((xs-xr)/(ys-yr)) + pi) - (atan((xq-xr)/(yq-yr)) + pi)
        angs = (atan((xp-xs)/(yp-ys)) + pi) - atan((xr-xs)/(yr-ys))
        angp  = atan((xq-xp)/(yq-yp)) - atan((xs-xp)/(ys-yp))
        dqr = sqrt((xr-xq)**2 + (yr-yq)**2)
        drs = sqrt((xs-xr)**2 + (ys-yr)**2)
        dsp = sqrt((xp-xs)**2 + (yp-ys)**2)
        dpq = sqrt((xq-xp)**2 + (yq-yp)**2)
        dirpq = atan((xq-xp)/(yq-yp))

        # Parametric model equations in matrix form
        obs = Matrix([angq, angr, angs, angp, dqr, drs, dsp, dpq, dirpq])
        par = Matrix([xq, yq, xr, yr, xs, ys])
        df = len(obs) - len(par)

        # Jacobian wrt unknown arameters
        j = obs.jacobian(par)

        # Jacobian evaluated at given values as A matrix
        a = j.subs({xp: 1000.000, yp: 1150.000,
                    xq: txq, yq: tyq,
                    xr: txr, yr: tyr,
                    xs: txs, ys: tys})

        # Computed observations L0
        l0 = obs.subs({xp: 1000.000, yp: 1150.000,
                       xq: txq, yq: tyq,
                       xr: txr, yr: tyr,
                       xs: txs, ys: tys})

        # Given observations L
        l = Matrix([angqq, angrr,
                    angss, angpp,
                    dqrr, drss,
                    dspp, dpqq,
                    dirpqq])

        # Misclosure vector w
        w = l0 - l

        # Wight matrix P
        p = diag(rad(4/3600)**-2, rad(4/3600)**-2,
                 rad(4/3600)**-2, rad(4/3600)**-2,
                 0.0027**-2, 0.0027**-2, 0.0027**-2, 0.0027**-2, rad(4/3600)**-2)

        # Matrix of coefficients of normal equations N and u
        n = a.T@p@a
        u = a.T@p@w

        # Vector of corrections d to aproximate arameters
        d = -n.inv()@u

        # Check stoping criteria
        correction = max(abs(d))
        
        # Print iteration information
        print(f'\nIteration {iteration + 1}:')
        print(f"Initial Coordinates of Q, R, S:")
        print(f"xP: {1000:.4f}m  yP: {1150:.4f}m")
        print(f"xQ: {txq:.4f}m  yQ: {tyq:.4f}m")
        print(f"xR: {txr:.4f}m  yR: {tyr:.4f}m")
        print(f"xS: {txs:.4f}m  yS: {tys:.4f}m")
        
        # Display corrections
        print(f'Correction to Initial Coordinates:\
        \ndxQ: {d[0]:.4f}m  dyQ: {d[1]:.4f}m\
        \ndxR: {d[2]:.4f}m  dyR: {d[3]:.4f}m\
        \ndxS: {d[4]:.4f}m  dyS: {d[5]:.4f}m')
        
        # Update initial coordinates with corrections
        txq += d[0]; tyq += d[1]
        txr += d[2]; tyr += d[3]
        txs += d[4]; tys += d[5]
        
        # Print adjusted coordinates
        print(f"Adjusted Coordinates of Q, R, S, T, U and W:")
        print(f"xP: {1000:.4f}m  yP: {1150:.4f}m")
        print(f"xQ: {txq:.4f}m   yQ: {tyq:.4f}m")
        print(f"xR: {txr:.4f}m   yR: {tyr:.4f}m")
        print(f"xS: {txs:.4f}m   yS: {tys:.4f}m")
        
        if correction < max_correction: # type: ignore
            print(
            f"\nStoping Criterion Met: Correction {correction:.4f}m is less than {max_correction:.4f}m"
                 )
            break
        
    # Store the final values
    a = a; l = l; w = w; p = p
    n = n; d = d; df = df
        
    return a, l, w, p, n, d, df

def postdj(a, l, w, p, n, d, df):
    # Residual vector v
    v = a@d+w

    # Aposteriori variance of unit weight s02
    s02 = v.T@p@v/df
    s02 = s02[0]

    # Cofactor of adjusted arameters qx
    qx = n.inv()

    # Covariance matrix of djusted arameters cx
    cx = s02*qx

    adjobs = l + v

    # Adjusted Observations
    print(f"Adjusted Observations:")
    print(f'Angle Q: {degtodms(deg(adjobs[0]))}')
    print(f'Angle R: {degtodms(deg(adjobs[1]))}')
    print(f'Angle S: {degtodms(deg(adjobs[2]))}')
    print(f'Angle P: {degtodms(deg(adjobs[3]))}')
    print(f"Distance QR: {adjobs[4]:.3f}m")
    print(f"Distance RS: {adjobs[5]:.3f}m")
    print(f"Distance SP: {adjobs[6]:.3f}m")
    print(f"Distance PQ: {adjobs[7]:.3f}m")
    print(f'Direction PQ: {degtodms(deg(adjobs[8]))}')
    
    print(f"\nStandard Deviation of Q, R, S:")
    print(f"SDxP: {0:.4f}m   SDyP: {0:.4f}m")
    print(f"SDxQ: {sqrt(cx[0,0]):.4f}m   SDyQ: {sqrt(cx[1,1]):.4f}m")
    print(f"SDxR: {sqrt(cx[2,2]):.4f}m   SDyR: {sqrt(cx[3,3]):.4f}m")
    print(f"SDxS: {sqrt(cx[4,4]):.4f}m   SDyS: {sqrt(cx[5,5]):.4f}m")
    
    print(f"\nApostriori Variance Factor:")
    print(f"S02 = {s02:.2f}")

In [7]:
# Perform traverse and djust the aproximate points
a, l, w, p, n, d, df = traverse(1177.000, 1180.000,
                                1250.000, 1390.000,
                                1037.000, 1290.000)


Iteration 1:
Initial Coordinates of Q, R, S:
xP: 1000.0000m  yP: 1150.0000m
xQ: 1177.0000m  yQ: 1180.0000m
xR: 1250.0000m  yR: 1390.0000m
xS: 1037.0000m  yS: 1290.0000m
Correction to Initial Coordinates:        
dxQ: 0.3489m  dyQ: 2.4664m        
dxR: 1.5051m  dyR: 3.4132m        
dxS: 0.6454m  dyS: 0.8308m
Adjusted Coordinates of Q, R, S, T, U and W:
xP: 1000.0000m  yP: 1150.0000m
xQ: 1177.3489m   yQ: 1182.4664m
xR: 1251.5051m   yR: 1393.4132m
xS: 1037.6454m   yS: 1290.8308m

Iteration 2:
Initial Coordinates of Q, R, S:
xP: 1000.0000m  yP: 1150.0000m
xQ: 1177.3489m  yQ: 1182.4664m
xR: 1251.5051m  yR: 1393.4132m
xS: 1037.6454m  yS: 1290.8308m
Correction to Initial Coordinates:        
dxQ: -0.0165m  dyQ: 0.0073m        
dxR: -0.0112m  dyR: 0.0053m        
dxS: 0.0028m  dyS: -0.0027m
Adjusted Coordinates of Q, R, S, T, U and W:
xP: 1000.0000m  yP: 1150.0000m
xQ: 1177.3324m   yQ: 1182.4737m
xR: 1251.4940m   yR: 1393.4186m
xS: 1037.6482m   yS: 1290.8281m

Iteration 3:
Initial Coordinates

In [3]:
v = a@d+w
print(degtodms(deg(v[3])))

0° 0' 0.29"


<img src="images/fig1.png" width="650">

In [9]:
# Carry out post djustment analysis
postdj(a, l, w, p, n, d, df)

Adjusted Observations:
Angle Q: 119° 44' 50"
Angle R: 45° 0' 4"
Angle S: 130° 35' 46"
Angle P: 64° 39' 20"
Distance QR: 223.602m
Distance RS: 237.181m
Distance SP: 145.774m
Distance PQ: 180.281m
Direction PQ: 79° 37' 22"

Standard Deviation of Q, R, S:
SDxP: 0.0000m   SDyP: 0.0000m
SDxQ: 0.0036m   SDyQ: 0.0053m
SDxR: 0.0088m   SDyR: 0.0085m
SDxS: 0.0052m   SDyS: 0.0035m

Apostriori Variance Factor:
S02 = 2.32
